In [4]:
from glob import glob
from tqdm import tqdm
import os
import json
import cv2
import numpy as np
from pathlib import Path
# from prettyprinter import cpprint, set_default_style

# pred_json_dirA = "./output/structure_result/test_A-pred.json_0702_91.130"
pred_json_dirA = "./output/structure_result/test_A_jpg480max_wireless-pred.json"
pred_json_dirB = "./output/structure_result/test_A_jpg480max-pred.json"
visualize_dirA = f"{pred_json_dirA}_visualize"
visualize_dirB = f"{pred_json_dirB}_visualize"

diff_save_dir = "./output/structure_result/diff_map"
if os.path.exists(diff_save_dir):
    os.system(f"rm -rf {diff_save_dir}")
Path(diff_save_dir).mkdir(parents=True, exist_ok=True)

for pred_json_path_A in tqdm(glob(os.path.join(pred_json_dirA, "*-pred.json"))):
    json_file = os.path.basename(pred_json_path_A)
    img_file = json_file.replace("-pred.json", ".jpg")
    img_id = img_file.split(".")[0]

    pred_json_path_B = os.path.join(pred_json_dirB, json_file)
    if not os.path.exists(pred_json_path_B):
        continue
    dataA = json.load(open(pred_json_path_A, "r"))
    dataB = json.load(open(os.path.join(pred_json_dirB, json_file), "r"))

    img_path_A = os.path.join(visualize_dirA, f"{img_id}.jpg")
    img_path_B = os.path.join(visualize_dirB, f"{img_id}.jpg")
    if not os.path.exists(img_path_A) or not os.path.exists(img_path_B):
        continue

    # 比较layout排布
    if dataA['layout'] != dataB['layout']:
        imgA = cv2.imread(img_path_A)
        imgB = cv2.imread(img_path_B)
        widthA, heightA = imgA.shape[1], imgA.shape[0]
        widthB, heightB = imgB.shape[1], imgB.shape[0]
        if widthA != widthB or heightA != heightB:
            imgA = cv2.resize(imgA, (widthB, heightB))
        image = np.concatenate([imgA, imgB], axis=1)
        cv2.imwrite(os.path.join(diff_save_dir, f"{img_id}.jpg"), image)


  2%|▏         | 84/5187 [00:00<00:06, 838.24it/s]

100%|██████████| 5187/5187 [00:08<00:00, 621.18it/s]
